# Advanced Querying Mongo

Importing libraries and setting up connection

In [122]:
from pymongo import MongoClient
import pandas as pd
import pymongo
client = MongoClient("mongodb://localhost:27017")
db = client.companies
colec= db.companies
res= colec.find()

### 1. All the companies whose name match 'Babelgum'. Retrieve only their `name` field.

In [123]:
# Your Code
query = {"name": "Babelgum"}
select = {"name" : True, "_id":False}
res=colec.find(query, select)

list(res)

[{'name': 'Babelgum'}]

### 2. All the companies that have more than 5000 employees. Limit the search to 20 companies and sort them by **number of employees**.

In [124]:
# Your Code
query = {"number_of_employees": {"$gt": 5000}}
select = {"name": True, "_id": False, "number_of_employees": True}

res = colec.find(query, select).sort("number_of_employees", -1).limit(5)
list(res)

[{'name': 'Siemens', 'number_of_employees': 405000},
 {'name': 'IBM', 'number_of_employees': 388000},
 {'name': 'Toyota', 'number_of_employees': 320000},
 {'name': 'PayPal', 'number_of_employees': 300000},
 {'name': 'Nippon Telegraph and Telephone Corporation',
  'number_of_employees': 227000}]

### 3. All the companies founded between 2000 and 2005, both years included. Retrieve only the `name` and `founded_year` fields.

In [125]:
# Your Code
query = {"$and": [{"founded_year": {"$gte": 2000}}, {"founded_year":{"$lte":2005}}]}
select = {"_id": False, "founded_year": True, "name": True}


res = colec.find(query, select).limit(5)
list(res)

[{'name': 'Wetpaint', 'founded_year': 2005},
 {'name': 'Zoho', 'founded_year': 2005},
 {'name': 'Digg', 'founded_year': 2004},
 {'name': 'Facebook', 'founded_year': 2004},
 {'name': 'Omnidrive', 'founded_year': 2005}]

### 4. All the companies that had a Valuation Amount of more than 100.000.000 and have been founded before 2010. Retrieve only the `name` and `ipo` fields.

In [126]:
# Your Code
query = {"$and": [{"ipo.valuation_amount":{"$gt": 100000000}}, {"founded_year":{"$lt":2010}}]}
select= {"name":True, "valuation_amount": True, "_id": False} 
res = colec.find(query, select).limit(5)
list(res)


[{'name': 'Facebook'},
 {'name': 'Twitter'},
 {'name': 'Yelp'},
 {'name': 'LinkedIn'},
 {'name': 'Amazon'}]

### 5. All the companies that have less than 1000 employees and have been founded before 2005. Order them by the number of employees and limit the search to 10 companies.

In [144]:
# Your Code

query={"$and": [{"number_of_employees":{"$lt": 1000}}, {"founded_year": {"$lt":2005}}]}
select={"name": True,"number_of_employees": True, "_id":False}
res = colec.find(query, select).sort("number_of_employees", pymongo.DESCENDING).limit(5)
list(res)

[{'name': 'Infinera Corporation', 'number_of_employees': 974},
 {'name': 'NorthPoint Communications Group', 'number_of_employees': 948},
 {'name': '888 Holdings', 'number_of_employees': 931},
 {'name': 'Forrester Research', 'number_of_employees': 903},
 {'name': 'Webmetrics', 'number_of_employees': 900}]

### 6. All the companies that don't include the `partners` field.

In [153]:
# Your Code
query = {"partners": {"$exists": False}}
select = {"name" : True, "_id":False, "partners":True}

res = colec.find(query, select)
list(res)

[]

### 7. All the companies that have a null type of value on the `category_code` field.

In [129]:
# Your Code

query = { "category_code": {"$type": "null"}}
select = {"name":True, "_id": False}

res = colec.find(query, select).limit(5)
list(res)

[{'name': 'Collective'},
 {'name': 'Snimmer'},
 {'name': 'KoolIM'},
 {'name': 'Level9 Media'},
 {'name': 'VidKing'}]

### 8. All the companies that have at least 100 employees but less than 1000. Retrieve only the `name` and `number of employees` fields.

In [130]:
# Your Code

query = {"$and": [{"number_of_employees": {"$gte":100}}, 
                  {"number_of_employees" : {"$lt":1000}}]}

select = {"name":True, "number_of_employees": True, "_id": False}
res = colec.find(query, select).limit(5)
list(res)

[{'name': 'AdventNet', 'number_of_employees': 600},
 {'name': 'AddThis', 'number_of_employees': 120},
 {'name': 'OpenX', 'number_of_employees': 305},
 {'name': 'LifeLock', 'number_of_employees': 644},
 {'name': 'Jajah', 'number_of_employees': 110}]

### 9. Order all the companies by their IPO price in a descending order.

In [131]:
# Your Code

query = {"ipo.valuation_amount": {"$gte": 0}}

select={"name":True, "ipo.valuation_amount":True, "_id": False}


res = colec.find(query, select).sort("ipo.valuation_amount", pymongo.DESCENDING).limit(5) #no acabo de pillar el sort de mongo
list(res)


[{'name': 'GREE', 'ipo': {'valuation_amount': 108960000000.0}},
 {'name': 'Facebook', 'ipo': {'valuation_amount': 104000000000.0}},
 {'name': 'Amazon', 'ipo': {'valuation_amount': 100000000000.0}},
 {'name': 'Twitter', 'ipo': {'valuation_amount': 18100000000.0}},
 {'name': 'Groupon', 'ipo': {'valuation_amount': 12800000000.0}}]

### 10. Retrieve the 10 companies with more employees, order by the `number of employees`

In [160]:
# Your Code
#query = {"number_of_employees": {"$lt": 10000000000000000}} esto tira aunque no creo que sea la forma de hacerlo
select = {"name": True, "number_of_employees": True, "_id": False}
res = colec.find({}, select).sort("number_of_employees", pymongo.DESCENDING).limit(10) 
list(res)

[{'name': 'Siemens', 'number_of_employees': 405000},
 {'name': 'IBM', 'number_of_employees': 388000},
 {'name': 'Toyota', 'number_of_employees': 320000},
 {'name': 'PayPal', 'number_of_employees': 300000},
 {'name': 'Nippon Telegraph and Telephone Corporation',
  'number_of_employees': 227000},
 {'name': 'Samsung Electronics', 'number_of_employees': 221726},
 {'name': 'Accenture', 'number_of_employees': 205000},
 {'name': 'Tata Consultancy Services', 'number_of_employees': 200300},
 {'name': 'Flextronics International', 'number_of_employees': 200000},
 {'name': 'Safeway', 'number_of_employees': 186000}]

### 11. All the companies founded on the second semester of the year. Limit your search to 1000 companies.

In [166]:
# Your Code
query = {"founded_month":{"$gte":6}}  
select = {"founded_month":True, "name": True, "_id" : False} 

res = colec.find(query, select).limit(10) #ponia 1000 pero por evitar el scroll innecesario
list(res)

[{'name': 'Wetpaint', 'founded_month': 10},
 {'name': 'Zoho', 'founded_month': 9},
 {'name': 'Digg', 'founded_month': 10},
 {'name': 'Omnidrive', 'founded_month': 11},
 {'name': 'Postini', 'founded_month': 6},
 {'name': 'Geni', 'founded_month': 6},
 {'name': 'Fox Interactive Media', 'founded_month': 6},
 {'name': 'eBay', 'founded_month': 9},
 {'name': 'Joost', 'founded_month': 10},
 {'name': 'Plaxo', 'founded_month': 11}]

### 12. All the companies founded before 2000 that have an acquisition amount of more than 10.000.00

In [170]:
# Your Code
query = {"$and":[{"founded_year": {"$lt": 2000}}, 
                 {"acquisition.price_amount": {"$gt": 10000000}}]}

select = {"name": True, "acquisition.price_amount": True, "_id": False}

res = colec.find(query, select).limit(5)
list(res)


[{'name': 'Postini', 'acquisition': {'price_amount': 625000000}},
 {'name': 'SideStep', 'acquisition': {'price_amount': 180000000}},
 {'name': 'Recipezaar', 'acquisition': {'price_amount': 25000000}},
 {'name': 'PayPal', 'acquisition': {'price_amount': 1500000000}},
 {'name': 'Snapfish', 'acquisition': {'price_amount': 300000000}}]

### 13. All the companies that have been acquired after 2010, order by the acquisition amount, and retrieve only their `name` and `acquisition` field.

In [172]:
# Your Code

query = {"acquisition.acquired_year": {"$gt":2010}}
select = {"acquisition":True, "name": True, "_id": False}

res = colec.find(query, select).limit(5)
list(res)

[{'name': 'Wetpaint',
  'acquisition': {'price_amount': 30000000,
   'price_currency_code': 'USD',
   'term_code': 'cash_and_stock',
   'source_url': 'http://allthingsd.com/20131216/viggle-tries-to-bulk-up-its-social-tv-business-by-buying-wetpaint/?mod=atdtweet',
   'source_description': ' Viggle Tries to Bulk Up Its Social TV Business by Buying Wetpaint',
   'acquired_year': 2013,
   'acquired_month': 12,
   'acquired_day': 16,
   'acquiring_company': {'name': 'Viggle', 'permalink': 'viggle'}}},
 {'name': 'Digg',
  'acquisition': {'price_amount': 500000,
   'price_currency_code': 'USD',
   'term_code': None,
   'source_url': 'http://techcrunch.com/2012/07/12/betaworks-acquires-digg/',
   'source_description': 'Betaworks Acquires Digg (TechCrunch)',
   'acquired_year': 2012,
   'acquired_month': 7,
   'acquired_day': 12,
   'acquiring_company': {'name': 'betaworks', 'permalink': 'betaworks'}}},
 {'name': 'Geni',
  'acquisition': {'price_amount': None,
   'price_currency_code': 'USD',
 

### 14. Order the companies by their `founded year`, retrieving only their `name` and `founded year`.

In [176]:
# Your Code

query = {}
select = {"name":True, "founded_year": True, "_id":False}

res = colec.find(query, select).sort("founded_year", pymongo.DESCENDING).limit(5)
list(res)

[{'name': 'Advaliant', 'founded_year': 2013},
 {'name': 'Fixya', 'founded_year': 2013},
 {'name': 'iBazar', 'founded_year': 2013},
 {'name': 'Fluc', 'founded_year': 2013},
 {'name': 'Wamba', 'founded_year': 2013}]

### 15. All the companies that have been founded on the first seven days of the month, including the seventh. Sort them by their `acquisition price` in a descending order. Limit the search to 10 documents.

In [177]:
# Your Code
query = {"founded_day": {"$lt":7}}
select = {"name": True, "founded_day": True, "acquisition.price_amount": True, "_id":False}

res = colec.find(query, select).sort("acquisition.price_amount", pymongo.DESCENDING).limit(5)
list(res)


[{'name': 'Netscape',
  'founded_day': 4,
  'acquisition': {'price_amount': 4200000000.0}},
 {'name': 'PayPal',
  'founded_day': 1,
  'acquisition': {'price_amount': 1500000000}},
 {'name': 'Zappos',
  'founded_day': 1,
  'acquisition': {'price_amount': 1200000000}},
 {'name': 'Alibaba',
  'founded_day': 1,
  'acquisition': {'price_amount': 1000000000}},
 {'name': 'Postini',
  'founded_day': 2,
  'acquisition': {'price_amount': 625000000}}]

### 16. All the companies on the 'web' `category` that have more than 4000 employees. Sort them by the amount of employees in ascending order.

In [179]:
# Your Code
query = {"$and":[{"category_code": "web"},{"number_of_employees":{"$gt":4000}}]}
select= {"name":True, "category_code":True, "number_of_employees":True, "_id":False}


res = colec.find(query, select).sort("number_of_employees", pymongo.ASCENDING).limit(5)
list(res)

[{'name': 'Expedia', 'category_code': 'web', 'number_of_employees': 4400},
 {'name': 'AOL', 'category_code': 'web', 'number_of_employees': 8000},
 {'name': 'Webkinz', 'category_code': 'web', 'number_of_employees': 8657},
 {'name': 'Los Angeles Times Media Group',
  'category_code': 'web',
  'number_of_employees': 10000},
 {'name': 'Rakuten', 'category_code': 'web', 'number_of_employees': 10000}]

### 17. All the companies whose acquisition amount is more than 10.000.000, and currency is 'EUR'.

In [180]:
# Your Code
query = {"$and":[{"acquisition.price_amount":{"$gt": 10000000}},
                 {"acquisition.price_currency_code": "EUR"}]}
select = {"name": True, "acquisition.price_amount": True, "_id":False}

res = colec.find(query, select).limit(5)
list(res)

[{'name': 'ZYB', 'acquisition': {'price_amount': 31500000}},
 {'name': 'Apertio', 'acquisition': {'price_amount': 140000000}},
 {'name': 'Greenfield Online', 'acquisition': {'price_amount': 40000000}},
 {'name': 'Webedia', 'acquisition': {'price_amount': 70000000}},
 {'name': 'Wayfinder', 'acquisition': {'price_amount': 24000000}}]

### 18. All the companies that have been acquired on the first trimester of the year. Limit the search to 10 companies, and retrieve only their `name` and `acquisition` fields.

In [140]:
# Your Code

# Bonus
### 19. All the companies that have been founded between 2000 and 2010, but have not been acquired before 2011.

In [141]:
# Your Code

### 20. All the companies that have been 'deadpooled' after the third year.

In [142]:
# Your Code